In [1]:
import auxiliary as aux

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Input, BatchNormalization


valid_true, valid = aux.train_valid_split(aux.data, frac=0.0125, seed=19)

In [ ]:
# add_cols = ['F_1_7', 'F_1_12', 'F_1_13'] + ['F_3_19', 'F_3_21']
# pred_cols = [*[f'F_1_{n}' for n in range(15) if n not in (7, 12, 13)],
#              *[f'F_3_{n}' for n in range(25) if n not in (19, 21)]]

## tests

In [6]:
# %%time
# predicted = aux.ImputeHelper(
#     aux.Step(aux.predictor, aux.subcol[4], estimator=LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29)),
#     aux.Step(aux.transformer, 'all', initiator=SimpleImputer()),
# ).run(valid, validate_on=valid_true)

Final validation: 100%|██████████| 55/55 [00:00<00:00, 456.74it/s]


Final validation score: 0.8783267769946143
Overall final score: 0.10180127772818622
CPU times: user 56min 41s, sys: 55.6 s, total: 57min 36s
Wall time: 8min 18s


In [7]:
# %%time
# # only 4th subset
# predicted = aux.ImputeHelper(
#     aux.Step(aux.predictor, aux.subcol[4], estimator=LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29)),
# ).run(valid, validate_on=valid_true)

Final validation: 100%|██████████| 15/15 [00:00<00:00, 479.14it/s]

Final validation score: 0.6478440895126936
Overall final score: 0.060344738210697677
CPU times: user 52min 14s, sys: 45.8 s, total: 52min 59s
Wall time: 7min 30s


__appempts__<br>
is improve if less than<br>
Final validation score: `0.64784`<br>
Overall final score: `0.060344`

In [6]:
# predicted = aux.ImputeHelper(
#     aux.Step(aux.predictor, aux.subcol[4], estimator=XGBRegressor(random_state=7, n_jobs=-1, n_estimators=1000, max_leaves=29)),
# ).run(valid, validate_on=valid_true)

Final validation: 100%|██████████| 15/15 [00:00<00:00, 346.49it/s]


Final validation score: 0.7726241830803292
Overall final score: 0.0714207626651449


In [8]:
predicted = aux.ImputeHelper(
    aux.Step(aux.predictor, aux.subcol[4],
             initiator=LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29),
             estimator=LGBMRegressor(random_state=11, n_jobs=-1, n_estimators=5000, num_leaves=37)),
).run(valid, validate_on=valid_true)
# Final validation score: 0.5392914307519079
# Overall final score: 0.04965443938319474

Using `@parameter.initiator.allowed('any')` is redundant


Final validation: 100%|██████████| 15/15 [00:00<00:00, 470.49it/s]

Final validation score: 0.5392914307519079
Overall final score: 0.04965443938319474


try below

In [2]:
# %%time
# predicted = aux.ImputeHelper(
#     aux.Step(aux.transformer, aux.subcol[4], initiator=IterativeImputer(LGBMRegressor(n_jobs=-1, n_estimators=5000, num_leaves=37), random_state=11, max_iter=15)),
# ).run(valid, validate_on=valid_true)

/home/avagadro/anaconda3/envs/jupyter_default/lib/python3.9/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
Final validation: 100%|██████████| 15/15 [00:00<00:00, 463.56it/s]


Final validation score: 0.5731784815583407
Overall final score: 0.05217318235972474
CPU times: user 1d 2h 29min 11s, sys: 26min 28s, total: 1d 2h 55min 40s
Wall time: 3h 46min 10s


In [13]:
# predicted = aux.ImputeHelper(
#     aux.Step(aux.mice, aux.subcol[4], epochs=3,  seed=17,
#              initiator=LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29),
#              estimator=LGBMRegressor(n_jobs=-1, n_estimators=5000, num_leaves=37)),
# ).run(valid, validate_on=valid_true)

Final validation: 100%|██████████| 15/15 [00:00<00:00, 491.99it/s]


Final validation score: 0.5296390782289352
Overall final score: 0.048593891495186274


In [2]:
%%time
predicted = aux.ImputeHelper(
    aux.Step(aux.predictor, aux.subcol[4],
             initiator=IterativeImputer(LGBMRegressor(n_jobs=-1, n_estimators=5000, num_leaves=29), random_state=11, max_iter=5),
             estimator=LGBMRegressor(random_state=11, n_jobs=-1, n_estimators=5000, num_leaves=37)),
).run(valid, validate_on=valid_true)

Using `@parameter.initiator.allowed('any')` is redundant


/home/avagadro/anaconda3/envs/jupyter_default/lib/python3.9/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
Final validation: 100%|██████████| 15/15 [00:00<00:00, 393.42it/s]


Final validation score: 0.5264524264802676
Overall final score: 0.04831689151912407
CPU times: user 11h 9min 39s, sys: 9min 23s, total: 11h 19min 2s
Wall time: 1h 32min 49s


In [3]:
predicted = aux.ImputeHelper(
    aux.Step(aux.mice, aux.subcol[4], epochs=3,  seed=17,
             initiator=IterativeImputer(LGBMRegressor(n_jobs=-1, n_estimators=5000, num_leaves=29), random_state=11, max_iter=5),
             estimator=LGBMRegressor(n_jobs=-1, n_estimators=5000, num_leaves=37)),
).run(valid, validate_on=valid_true)

/home/avagadro/anaconda3/envs/jupyter_default/lib/python3.9/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
Final validation: 100%|██████████| 15/15 [00:00<00:00, 462.62it/s]


Final validation score: 0.540339984361973
Overall final score: 0.049523684565388866


In [ ]:
# # NN approach
# tf.random.set_seed(11)

# model = Sequential([
#     Input(shape=(25, )),
#     Dense(256, activation='tanh'),
#     Dense(256, activation='tanh'),
#     Dense(15, activation='tanh'),
# ])
# model.compile(optimizer='adam', loss='mse', metrics='mse')

# predicted = aux.ImputeHelper(
#     aux.Step(aux.transformer, aux.subcol[1] + aux.subcol[3], initiator=SimpleImputer()),
#     aux.Step(aux.onestep_neural, aux.subcol[3] + aux.subcol[4], initiator=SimpleImputer(), model=model, epochs=3),
# ).run(valid, validate_on=valid_true)
# # it is an improve if is less than
# # Final validation score: 0.8783267769946155
# # Overall final score: 0.10180127772818637

## temp

In [ ]:
predicted.isna().any().any()

In [ ]:
# default
# Final validation score: 0.8783267769946155
# Overall final score: 0.10180127772818637